# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f5b10d4c970>
├── 'a' --> tensor([[-0.2873, -1.2421,  0.1464],
│                   [-0.6610, -0.1442, -1.5657]])
└── 'x' --> <FastTreeValue 0x7f5b10d4cf70>
    └── 'c' --> tensor([[ 0.4665,  2.0195, -0.1632, -0.3297],
                        [-0.6056,  0.9347, -1.3495, -1.7604],
                        [-0.1045,  0.1545, -0.3433, -2.4865]])

In [4]:
t.a

tensor([[-0.2873, -1.2421,  0.1464],
        [-0.6610, -0.1442, -1.5657]])

In [5]:
%timeit t.a

58.6 ns ± 0.0484 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f5b10d4c970>
├── 'a' --> tensor([[-0.0714, -0.3406, -0.2219],
│                   [ 1.2379, -1.1599, -0.6488]])
└── 'x' --> <FastTreeValue 0x7f5b10d4cf70>
    └── 'c' --> tensor([[ 0.4665,  2.0195, -0.1632, -0.3297],
                        [-0.6056,  0.9347, -1.3495, -1.7604],
                        [-0.1045,  0.1545, -0.3433, -2.4865]])

In [7]:
%timeit t.a = new_value

58.8 ns ± 0.0317 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.2873, -1.2421,  0.1464],
               [-0.6610, -0.1442, -1.5657]]),
    x: Batch(
           c: tensor([[ 0.4665,  2.0195, -0.1632, -0.3297],
                      [-0.6056,  0.9347, -1.3495, -1.7604],
                      [-0.1045,  0.1545, -0.3433, -2.4865]]),
       ),
)

In [10]:
b.a

tensor([[-0.2873, -1.2421,  0.1464],
        [-0.6610, -0.1442, -1.5657]])

In [11]:
%timeit b.a

55.1 ns ± 0.0546 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.2406,  0.8164,  0.9999],
               [-1.1155, -0.9031,  0.1904]]),
    x: Batch(
           c: tensor([[ 0.4665,  2.0195, -0.1632, -0.3297],
                      [-0.6056,  0.9347, -1.3495, -1.7604],
                      [-0.1045,  0.1545, -0.3433, -2.4865]]),
       ),
)

In [13]:
%timeit b.a = new_value

479 ns ± 0.303 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

830 ns ± 0.852 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 17.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 617 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 708 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f5a5bf86d30>
├── 'a' --> tensor([[[-0.2873, -1.2421,  0.1464],
│                    [-0.6610, -0.1442, -1.5657]],
│           
│                   [[-0.2873, -1.2421,  0.1464],
│                    [-0.6610, -0.1442, -1.5657]],
│           
│                   [[-0.2873, -1.2421,  0.1464],
│                    [-0.6610, -0.1442, -1.5657]],
│           
│                   [[-0.2873, -1.2421,  0.1464],
│                    [-0.6610, -0.1442, -1.5657]],
│           
│                   [[-0.2873, -1.2421,  0.1464],
│                    [-0.6610, -0.1442, -1.5657]],
│           
│                   [[-0.2873, -1.2421,  0.1464],
│                    [-0.6610, -0.1442, -1.5657]],
│           
│                   [[-0.2873, -1.2421,  0.1464],
│                    [-0.6610, -0.1442, -1.5657]],
│           
│                   [[-0.2873, -1.2421,  0.1464],
│                    [-0.6610, -0.1442, -1.5657]]])
└── 'x' --> <FastTreeValue 0x7f5a5bf86d60>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.8 µs ± 68.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f5a5bf863a0>
├── 'a' --> tensor([[-0.2873, -1.2421,  0.1464],
│                   [-0.6610, -0.1442, -1.5657],
│                   [-0.2873, -1.2421,  0.1464],
│                   [-0.6610, -0.1442, -1.5657],
│                   [-0.2873, -1.2421,  0.1464],
│                   [-0.6610, -0.1442, -1.5657],
│                   [-0.2873, -1.2421,  0.1464],
│                   [-0.6610, -0.1442, -1.5657],
│                   [-0.2873, -1.2421,  0.1464],
│                   [-0.6610, -0.1442, -1.5657],
│                   [-0.2873, -1.2421,  0.1464],
│                   [-0.6610, -0.1442, -1.5657],
│                   [-0.2873, -1.2421,  0.1464],
│                   [-0.6610, -0.1442, -1.5657],
│                   [-0.2873, -1.2421,  0.1464],
│                   [-0.6610, -0.1442, -1.5657]])
└── 'x' --> <FastTreeValue 0x7f5a5bf86dc0>
    └── 'c' --> tensor([[ 0.4665,  2.0195, -0.1632, -0.3297],
                        [-0.6056,  0.9347, -1.3495, -1.7604],
                 

In [23]:
%timeit t_cat(trees)

30.3 µs ± 32.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.2 µs ± 72 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.2873, -1.2421,  0.1464],
                [-0.6610, -0.1442, -1.5657]],
       
               [[-0.2873, -1.2421,  0.1464],
                [-0.6610, -0.1442, -1.5657]],
       
               [[-0.2873, -1.2421,  0.1464],
                [-0.6610, -0.1442, -1.5657]],
       
               [[-0.2873, -1.2421,  0.1464],
                [-0.6610, -0.1442, -1.5657]],
       
               [[-0.2873, -1.2421,  0.1464],
                [-0.6610, -0.1442, -1.5657]],
       
               [[-0.2873, -1.2421,  0.1464],
                [-0.6610, -0.1442, -1.5657]],
       
               [[-0.2873, -1.2421,  0.1464],
                [-0.6610, -0.1442, -1.5657]],
       
               [[-0.2873, -1.2421,  0.1464],
                [-0.6610, -0.1442, -1.5657]]]),
    x: Batch(
           c: tensor([[[ 0.4665,  2.0195, -0.1632, -0.3297],
                       [-0.6056,  0.9347, -1.3495, -1.7604],
                       [-0.1045,  0.1545, -0.3433, -2.4865]],
         

In [26]:
%timeit Batch.stack(batches)

78.2 µs ± 278 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.2873, -1.2421,  0.1464],
               [-0.6610, -0.1442, -1.5657],
               [-0.2873, -1.2421,  0.1464],
               [-0.6610, -0.1442, -1.5657],
               [-0.2873, -1.2421,  0.1464],
               [-0.6610, -0.1442, -1.5657],
               [-0.2873, -1.2421,  0.1464],
               [-0.6610, -0.1442, -1.5657],
               [-0.2873, -1.2421,  0.1464],
               [-0.6610, -0.1442, -1.5657],
               [-0.2873, -1.2421,  0.1464],
               [-0.6610, -0.1442, -1.5657],
               [-0.2873, -1.2421,  0.1464],
               [-0.6610, -0.1442, -1.5657],
               [-0.2873, -1.2421,  0.1464],
               [-0.6610, -0.1442, -1.5657]]),
    x: Batch(
           c: tensor([[ 0.4665,  2.0195, -0.1632, -0.3297],
                      [-0.6056,  0.9347, -1.3495, -1.7604],
                      [-0.1045,  0.1545, -0.3433, -2.4865],
                      [ 0.4665,  2.0195, -0.1632, -0.3297],
                      [-0.6056,  

In [28]:
%timeit Batch.cat(batches)

143 µs ± 374 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

306 µs ± 530 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
